In [1]:
import torch
from torch.autograd import Variable
import torch.nn.functional as Function

import matplotlib.pyplot as plt

# Data 

class Data(object):
    def __init__(self):
        # unsqueeze 一維轉成二維 [1,2,3] -> [[1,2,3]]  (3) -> (3,1)
        self.x = torch.unsqueeze(torch.linspace(-1,1,100),dim=1) 
        self.y = self.x.pow(2) + 0.2*torch.rand(self.x.size())

        self.x , self.y = Variable(self.x) , Variable(self.y)
        
    def get_data(self):
        return self.x , self.y

In [2]:
config = {
    'epoch':500,
    'lr': 0.001
}

In [3]:
data = Data()

x,y = data.get_data()

In [4]:
class Net(torch.nn.Module):
    def __init__(self,n_feature=1,n_hidden=1,n_label=1):
        super(Net,self).__init__()
        
        # Define Network
        # 這樣不代表已經建好神經網路
        self.hidden_layer = torch.nn.Linear(n_feature,n_hidden)
        self.output_layer = torch.nn.Linear(n_hidden,n_label)
        
        
    def forward(self,x):
        x = Function.relu(self.hidden_layer(x))
        x = self.output_layer(x)
        
        return x

In [5]:

net = Net(1,10,1)

print(net) # 查看結構


Net (
  (hidden_layer): Linear (1 -> 10)
  (output_layer): Linear (10 -> 1)
)


In [6]:
# 保存整個 model
def save(model,step):
    torch.save(model,"model_{}.pkl".format(step))

def restore_model(file_name):
    return torch.load(file_name)
    
# 只保存 model 參數
def save_parameters(model,step):
    torch.save(model.state_dict(),"model_para_{}.pkl".format(step))

def restore_model_paras(file_name):
    
    model = Net(1,10,1)
    model.load_state_dict( torch.load_state_dict(file_name))

In [8]:
# 給 optimizer 全部的參數 -> net.parameters()

optimizer = torch.optim.Adam(net.parameters(),lr=config['lr'])
loss_func = torch.nn.MSELoss()

net.train() # 設置 model 為 trainning mode

for step in range(config['epoch']+):
    out = net(x)
    loss = loss_func(out,y)
    
    if step%50 == 0:
        print("Epoch[{}/{}] loss:{}".format(step,config['epoch'],loss.data.numpy()))
        save(net,step)
    
    optimizer.zero_grad() # 初始梯度，上一次保留的梯度刪除
    loss.backward()
    optimizer.step() # 優化

Epoch[0/500] loss:[0.22226796]
Epoch[50/500] loss:[0.09225678]
Epoch[100/500] loss:[0.06454952]
Epoch[150/500] loss:[0.0552843]
Epoch[200/500] loss:[0.04712631]
Epoch[250/500] loss:[0.03947677]
Epoch[300/500] loss:[0.03241213]
Epoch[350/500] loss:[0.02618444]
Epoch[400/500] loss:[0.02096819]
Epoch[450/500] loss:[0.01678337]


/home/sin/anaconda3/envs/pytorch_learn/lib/python3.6/site-packages/torch/serialization.py:147: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [ ]:
restore_model("")